# Sub Function 1: Convert PDF to json file split on pages

In [1]:
import os

os.environ["Azure_BlobStorage_IngestionContainer"] = "ingestion-documents"
os.environ["Azure_BlobStorage_JsonContainer"] = "json-documents"
os.environ["Azure_BlobStorage_VectorContainer"] = "vector-documents"

In [2]:
import json
import os
from langchain.document_loaders import PyMuPDFLoader
from cloudpathlib.azure import AzureBlobPath, AzureBlobClient

file_name = "apexxnar-epar-product-information_nl.pdf"

client = AzureBlobClient(connection_string=os.environ.get("Azure_BlobStorage_ConnectionString"))
path = AzureBlobPath("az://" + os.environ.get("Azure_BlobStorage_IngestionContainer") + "/" + file_name, client=client)

with path.open(mode="r") as file:
    loader = PyMuPDFLoader(file.name)
    data = loader.load()
    json_output = json.dumps([d.dict() for d in data])
    jsonFile = AzureBlobPath("az://" + os.environ.get("Azure_BlobStorage_JsonContainer") + "/" + file_name.replace(".pdf",".json"), client=client)
    jsonFile.write_text(json_output)

# Sub Function 2: Tokenize json file and store as FAISS index

In [4]:
import json
import shutil
from pathlib import Path
from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from cloudpathlib.azure import AzureBlobPath, AzureBlobClient

file_name = "apexxnar-epar-product-information_nl.json"

client = AzureBlobClient(connection_string=os.environ.get("Azure_BlobStorage_ConnectionString"))
path = AzureBlobPath("az://" + os.environ.get("Azure_BlobStorage_JsonContainer") + "/" + file_name, client=client)
vectorPath = AzureBlobPath("az://" + os.environ.get("Azure_BlobStorage_VectorContainer") + "/" + path.stem, client=client)

tempPath = Path(path.stem)
tempPath.mkdir(exist_ok=True)

docs = [Document.parse_obj(d) for d in json.loads(path.read_text())]

embeddings = OpenAIEmbeddings(chunk_size=1, deployment="text-embedding-ada-002", model="text-embedding-ada-002")
faiss = FAISS.from_documents(docs, embeddings)

# TODO I have to save to this tempPath because I can't get it to work with the vectorPath directly.
faiss.save_local(tempPath, "index")

vectorPath.upload_from(tempPath)
shutil.rmtree(tempPath)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2022-12-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2022-12-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Crea